# HTTP Redirection Analysis
- This notebook demonstrates the basic use of `http_redirections` table by finding the most commonly occuring redirections.
- Partly based on https://github.com/mozilla/openwpm-crawler/blob/master/analysis/Sample%20Analysis.ipynb


In [1]:
import re
import json
import sqlite3
import pandas as pd
from collections import defaultdict

In [3]:
# import some analysis utilities from https://github.com/englehardt/crawl_utils
import sys
sys.path.append('./crawl_utils/')
import domain_utils as du

In [4]:
# use the sample sqlite for the 2018-06 stateless crawl
DB = 'sample_2018-06_1m_stateless_census_crawl.sqlite'

### Load the redirects and requests

In [5]:
# Load the data
con = sqlite3.connect(DB)
con.row_factory = sqlite3.Row
cur = con.cursor()
redirects = pd.read_sql_query("SELECT old_channel_id, new_channel_id, visit_id FROM http_redirects"
                              " WHERE old_channel_id IS NOT NULL AND is_sts_upgrade=0;", con)
requests = pd.read_sql_query("SELECT url, channel_id FROM http_requests;", con)


In [6]:
# build a map of channel_id to request url
channel_id_to_url_map = dict(zip(requests.channel_id, requests.url))

In [7]:
redirects["old_url"] = redirects["old_channel_id"].map(lambda x: channel_id_to_url_map.get(x, None))
redirects["new_url"] = redirects["new_channel_id"].map(lambda x: channel_id_to_url_map.get(x, None))

In [8]:
# Eliminate redirections that don't have a corresponding request in the http_requests table
redirects = redirects[~redirects.new_url.isnull()]

In [9]:
redirects['old_ps1'] = redirects['old_url'].apply(du.get_ps_plus_1)
redirects['new_ps1'] = redirects['new_url'].apply(du.get_ps_plus_1)

In [10]:
# only count redirections between different PS+1's
redirects = redirects[redirects.old_ps1!=redirects.new_ps1]

In [11]:
# only count a (src-dst) pair once on a website
redirects.drop_duplicates(subset=["visit_id", "old_ps1", "new_ps1"], inplace=True)

In [12]:
redirects.head()

,old_channel_id,new_channel_id,visit_id,old_url,new_url,old_ps1,new_ps1
29,{bf5b720e-3b7d-4c24-850d-f9ca2e32771f},{651404cf-6f90-4b2e-8d20-f25ee94e5fca},3,https://facebook.com/security/hsts-pixel.gif?c...,https://fbcdn.net/security/hsts-pixel.gif?c=2.5,facebook.com,fbcdn.net
38,{023b1347-20c6-4491-bdac-ff132a905586},{edd381eb-b452-4540-b000-174e2b4974b8},8,https://cm.g.doubleclick.net/pixel?google_nid=...,https://analytics.twitter.com/tpm?pid=969&puid...,doubleclick.net,twitter.com
41,{651404cf-6f90-4b2e-8d20-f25ee94e5fca},{6ba116ef-16f5-4c16-8bd1-4a1d962faf5b},3,https://fbcdn.net/security/hsts-pixel.gif?c=2.5,https://fbsbx.com/security/hsts-pixel.gif?c=5,fbcdn.net,fbsbx.com
44,{80fd6229-1375-47b3-affd-56e2463e0fee},{9c920524-7670-4a81-805f-4a54e2dba5f5},10,http://mat1.qq.com/www/images/ind36.gif,http://mat1.gtimg.com/www/images/ind36.gif,qq.com,gtimg.com
57,{6b874783-5896-49c0-bb8a-f4d95167884d},{d55bedf4-3440-4019-82d4-d045f0726101},9,https://bh.contextweb.com/bh/rtset?pid=557477&...,https://s.amazon-adsystem.com/ecm3?id=Q1Bt741B...,contextweb.com,amazon-adsystem.com


### Most frequent redirection domain pairs

In [13]:
redirects.groupby(['old_ps1', 'new_ps1']).size().reset_index(name='# sites').\
    sort_values(by=['# sites'], ascending=False).head(10)

,old_ps1,new_ps1,# sites
879,doubleclick.net,google.com,295
1097,google-analytics.com,doubleclick.net,246
1018,everesttech.net,rubiconproject.com,88
930,doubleclick.net,rubiconproject.com,87
297,adsrvr.org,rubiconproject.com,78
1284,mathtag.com,rubiconproject.com,72
1598,rubiconproject.com,yahoo.com,70
1961,yahoo.com,rubiconproject.com,68
1587,rubiconproject.com,doubleclick.net,68
1013,everesttech.net,openx.net,68


### Top redirection targets
- Top 10 domains that redirect to other domains.

In [14]:
redirects.groupby(['new_ps1']).size().reset_index(name='# sites').\
    sort_values(by=['# sites'], ascending=False).head(10)

,new_ps1,# sites
399,pubmatic.com,1242
176,doubleclick.net,765
367,openx.net,575
159,demdex.net,524
424,rubiconproject.com,490
108,bluekai.com,382
70,amazon-adsystem.com,326
235,google.com,309
286,krxd.net,272
101,bidswitch.net,229


### Top redirection sources
- Top 10 domains that are redirected to from other domains.

In [15]:
redirects.groupby(['old_ps1']).size().reset_index(name='counts').\
    sort_values(by=['counts'], ascending=False).head(10)

,old_ps1,counts
164,doubleclick.net,1134
45,adsrvr.org,608
179,everesttech.net,474
92,bidswitch.net,401
285,mathtag.com,395
35,adnxs.com,367
213,google-analytics.com,246
383,rubiconproject.com,220
493,yahoo.com,198
379,rlcdn.com,189
